In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
from flow_analysis_comps.processing.Fourier.OrientationSpaceManager import orientationSpaceManager
import colorcet  # noqa: F401
from flow_analysis_comps.data_structs.AOS_structs import OSFilterParams
from flow_analysis_comps.data_structs.kymographs import graphExtractConfig, kymoExtractConfig
from flow_analysis_comps.processing.graph_extraction.graph_extract import VideoGraphExtractor
from flow_analysis_comps.processing.kymographing.kymographer import KymographExtractor


%matplotlib widget

%load_ext autoreload
%autoreload 2

In [ ]:
video_root_folder = Path(
    r"/Users/simonvanstaalduine/AMOLF-SHIMIZU Dropbox/Simon van Staalduine/033"
    # r"G:\AMOLF_Data\AMOLF-SHIMIZU Dropbox\Simon van Staalduine\033"
)
graph_data = VideoGraphExtractor(video_root_folder, graphExtractConfig()).edge_data
kymograph_list = KymographExtractor(graph_data, kymoExtractConfig()).processed_kymographs

In [ ]:
AOS_params = OSFilterParams(space_frequency_center=.25, space_frequency_width=.3, x_spacing=kymograph_list[0].deltas.delta_x, y_spacing=kymograph_list[0].deltas.delta_t)
AOS_manager = orientationSpaceManager(AOS_params, kymograph_list[0].kymo_no_static)

angles_dict = AOS_manager.get_all_angles()

In [ ]:
from flow_analysis_comps.visualizing.kymographs import kymoVisualizer
from flow_analysis_comps.visualizing.AOSFilterVisualizer import AOSVisualizer

AOS_viz = AOSVisualizer(AOS_params)
AOS_viz.demo_image(AOS_manager)